In [11]:
import pandas as pd
import os

sondaje = 'DDH3964'
ruta_base = r'C:\Users\house\Desktop\Minera Los Pelambres\Muestrera\Mapeo\Reporte cutting'

In [ ]:
archivo_litologia = sondaje + '\\litologia.csv'
ruta_litologia = os.path.join(ruta_base, archivo_litologia)
df_litologia = pd.read_csv(ruta_litologia)
df_litologia

In [ ]:
archivo_volalt = sondaje + '\\volalt.csv'
ruta_volalt = os.path.join(ruta_base, archivo_volalt)
df_volalt = pd.read_csv(ruta_volalt)
df_volalt

In [ ]:
archivo_alteracion = sondaje + '\\alteracion.csv'
ruta_alteracion = os.path.join(ruta_base, archivo_alteracion)
df_alteracion = pd.read_csv(ruta_alteracion)
df_alteracion

In [ ]:
archivo_estructuras = sondaje + '\\estructuras.csv'
ruta_estructuras = os.path.join(ruta_base, archivo_estructuras)
df_estructuras = pd.read_csv(ruta_estructuras)
df_estructuras

ETL Mineralizacion

In [23]:
archivo_mineralizacion = sondaje + '\\mineralizacion.csv'
ruta_mineralizacion = os.path.join(ruta_base, archivo_mineralizacion)
df_mineralizacion = pd.read_csv(ruta_mineralizacion)
df_mineralizacion.drop(['Ident.', 'Zona mineral'], axis=1, inplace=True)
df_mineralizacion.fillna(0, inplace=True)
#Eliminar filas inválidad, las que tienen todos los valores 0
columnas = ['Desde', 'Hasta', '% Sulf', 'Py', 'Cpy', 'Cc', 'Bo', 'Mo']

#Eliminar las filas que contienen sólo 0
df_mineralizacion = df_mineralizacion.loc[~(df_mineralizacion[columnas] == 0).all(axis=1)]

#Reemplazar las filas cuando tienen valores 0 en las leyes
columnas_leyes = ['% Sulf', 'Py', 'Cpy', 'Cc', 'Bo', 'Mo']
df_mineralizacion.loc[(df_mineralizacion[columnas_leyes] == 0).all(axis=1), ['% Sulf', 'Py']] = 0.001

# Agregar una fila inicial si el primer intervalo no comienza desde 0
if df_mineralizacion['Desde'].iloc[0] > 0:
    first_row = {        
        'Desde': 0,
        'Hasta': df_mineralizacion['Desde'].iloc[0],        
        '% Sulf': 0.001,
        'Py': 0.001,
        'Cpy': 0.0,
        'Cc': 0.0,
        'Bo': 0.0,
        'Mo': 0.0
    }
    # Insertar la fila inicial en la primera posición
    df_mineralizacion = pd.concat([pd.DataFrame([first_row]), df_mineralizacion], ignore_index=True)

# Crear filas para cubrir los gaps entre las filas existentes
rows_to_update = []
for i in range(1, len(df_mineralizacion)):
    prev_hasta = df_mineralizacion.iloc[i - 1]['Hasta']
    curr_desde = df_mineralizacion.iloc[i]['Desde']
    
    if curr_desde > prev_hasta:  # Detectar gap
        gap_row = {            
            'Desde': prev_hasta,
            'Hasta': curr_desde,            
            '% Sulf': 0.001,
            'Py': 0.001,
            'Cpy': 0.0,
            'Cc': 0.0,
            'Bo': 0.0,
            'Mo': 0.0
        }
        rows_to_update.append((i, gap_row))
df_mineralizacion.to_csv(r'data\\mineralizacion.csv', index=False, header=True)
df_mineralizacion

,Desde,Hasta,% Sulf,Py,Cpy,Cc,Bo,Mo
0,0,52,0.001,0.001,0.0,0.0,0.0,0.0
1,52,54,0.001,0.001,0.0,0.0,0.0,0.0
2,54,56,0.001,0.001,0.0,0.0,0.0,0.0
3,56,58,0.001,0.001,0.0,0.0,0.0,0.0
4,58,60,0.001,0.001,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
409,868,870,0.001,0.001,0.0,0.0,0.0,0.0
410,870,872,0.001,0.001,0.0,0.0,0.0,0.0
411,872,874,0.001,0.001,0.0,0.0,0.0,0.0
412,874,876,0.001,0.001,0.0,0.0,0.0,0.0


In [ ]:
archivo_minzone = sondaje + '\\minzone.csv'
ruta_minzone = os.path.join(ruta_base, archivo_minzone)
df_minzone = pd.read_csv(ruta_minzone)
df_minzone

In [ ]:
archivo_pisotecho = sondaje + '\\pisotecho.csv'
ruta_pisotecho = os.path.join(ruta_base, archivo_pisotecho)
df_pisotecho = pd.read_csv(ruta_pisotecho)
df_pisotecho

In [ ]:
archivo_vetillas = sondaje + '\\vetillas.csv'
ruta_vetillas = os.path.join(ruta_base, archivo_vetillas)
df_vetillas = pd.read_csv(ruta_vetillas)
df_vetillas

In [ ]:
archivo_ug = sondaje + '\\UG.csv'
ruta_ug = os.path.join(ruta_base, archivo_ug)
df_ug = pd.read_csv(ruta_ug)
df_ug

In [71]:
df_vetillas = pd.read_csv(r"C:\Users\house\Desktop\vetillas.csv", delimiter=',')
df_vetillas.drop(['Registro'], axis=1, inplace=True)
df_vetillas[['T4', 'A', 'B', 'ED', 'D']] = df_vetillas[['T4', 'A', 'B', 'ED', 'D']].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
df_vetillas['Comentarios'] = df_vetillas['Comentarios'].fillna(0)
df_vetillas.head()

,Desde,Hasta,T4,A,B,ED,D,Comentarios
0,0,8.0,0,0,0,0,0,Fracturado
1,8,10.0,0,2,0,0,2,0
2,10,12.0,0,4,0,0,8,0
3,12,14.0,0,6,2,0,4,0
4,14,16.0,0,1,0,0,6,0


In [76]:
df_vetillas.to_csv(r"data\vetillas.csv", index=False, header=True)

Pega el siguiente código en  la línea de comandos

pyinstaller --onefile --hidden-import=comtypes.stream --hidden-import=comtypes --add-data "C:\Python\Mapeo\data\leyes.csv;." --add-data "C:\Python\Mapeo\data\vetillas.csv;." leyes.py